In [1]:
import pandas as pd
import numpy as np
# load libraries
import sys
from e2eml.regression import regression_blueprints
from e2eml.full_processing import postprocessing
from sklearn.model_selection import train_test_split
import gc

[nltk_data] Downloading package punkt to /home/thomas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/thomas/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/thomas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/thomas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
target = "score"

In [3]:
#comments = pd.read_csv("jigsaw_toxic_comments_to_score.csv")
#comments

In [4]:
df_train = pd.read_csv("jigsaw_toxic_validation_data.csv")
df_train

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,0,0,0,0,0
1,000103f0d9cfb60f,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember what page that's on?",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when your view completely contradicts the coverage in reliable sources, why should anyone care what you feel? You can't even give a consistent argument - is the opening only supposed to mention significant aspects, or the """"most significant"""" ones? \n\n""",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is a horrible thing you put on my talk page. 128.61.19.93,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for prostitution ring. - Crunch Captain.",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.,0,0,0,0,0,0


In [5]:
cat_mtpl = {'obscene': 0.16, 'toxic': 0.32, 'threat': 1.5, 
            'insult': 0.64, 'severe_toxic': 1.5, 'identity_hate': 1.5}

for category in cat_mtpl:
    df_train[category] = df_train[category] * cat_mtpl[category]

df_train['score'] = df_train.loc[:, 'toxic':'identity_hate'].mean(axis=1)

min_len = (df_train['score'] > 0).sum()  # len of toxic comments
df_y0_undersample = df_train[df_train['score'] == 0].sample(n=min_len, random_state=41)  # take non toxic comments
df_train = pd.concat([df_train[df_train['score'] > 0], df_y0_undersample])  # make new df

df_train

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,score
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,0.32,1.5,0.16,0.0,0.64,0.0,0.436667
12,0005c987bdfc9d4b,"Hey... what is it..\n@ | talk .\nWhat is it... an exclusive group of some WP TALIBANS...who are good at destroying, self-appointed purist who GANG UP any one who asks them questions abt their ANTI-SOCIAL and DESTRUCTIVE (non)-contribution at WP?\n\nAsk Sityush to clean up his behavior than issue me nonsensical warnings...",0.32,0.0,0.00,0.0,0.00,0.0,0.053333
16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming back! Tosser.",0.32,0.0,0.00,0.0,0.00,0.0,0.053333
42,001810bf8c45bf5f,"You are gay or antisemmitian? \n\nArchangel WHite Tiger\n\nMeow! Greetingshhh!\n\nUh, there are two ways, why you do erased my comment about WW2, that holocaust was brutally slaying of Jews and not gays/Gypsys/Slavs/anyone...\n\n1 - If you are anti-semitian, than shave your head bald and go to the skinhead meetings!\n\n2 - If you doubt words of the Bible, that homosexuality is a deadly sin, make a pentagram tatoo on your forehead go to the satanistic masses with your gay pals!\n\n3 - First and last warning, you fucking gay - I won't appreciate if any more nazi shwain would write in my page! I don't wish to talk to you anymore!\n\nBeware of the Dark Side!",0.32,0.0,0.16,0.0,0.64,1.5,0.436667
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",0.32,0.0,0.16,0.0,0.64,0.0,0.186667
...,...,...,...,...,...,...,...,...,...
137462,df957e1303ef8c68,"""\n\nRead more\nYou can read more by going to anomtalk.com except with """"n"""" instead of """"m"""".""",0.00,0.0,0.00,0.0,0.00,0.0,0.000000
82635,dd0b9714b24c3c66,Sounds like a deal. Thanks for hearing me out.,0.00,0.0,0.00,0.0,0.00,0.0,0.000000
149206,58e756534c67c8e7,"Bone to pick \n\nWhat makes you the rulemaker on all things bears related? Bad and Good go into articles. Get a life and stay off wikipedia for a good month or two. People like you are the reason that wikipedia is a joke. Too much of your personal feelings goes into making sure nothing you don't agree with gets put into an article. Last I checked your job wasn't encyclopedia writer, so let others ideas be put into articles.214.13.192.187",0.00,0.0,0.00,0.0,0.00,0.0,0.000000
87807,eade5e302541c3a9,"""\n\n Semi-vegetarianism section \n\nI like Flyer22's intention in that one's recent edits, though I find the particulars a little unclear. Here is my proposal for the section ( ):\n===Semi-vegetarian diets===\nSemi-vegetarian diets primarily consist of vegetarian foods, though occasional exceptions are made for some non-vegetarian foods, including fish, poultry and even red meat. These diets may be followed by those who choose to reduce the amount of animal flesh consumed as a way of transitioning to a vegetarian diet, or for health, environmental, or other reasons. The term """"semi-vegetarian"""" is contested by most vegetarian groups, who believe that vegetarianism must exclude all animal flesh. Many individuals describe themselves as """"vegetarian"""" while practicing a semi-vegetarian diet. Semi-vegetarian diets include: \n Flexitarianism: A diet that consists primarily of vegetarian food, but includes occasional exceptions such as red meat.\n Pescetarianism: A diet that is mainly ...",0.00,0.0,0.00,0.0,0.00,0.0,0.000000


In [6]:
df_train = df_train.sample(frac=1, random_state=5000)
train_df = df_train.head(30000).copy()
val_df = df_train.tail(2000).copy()

In [7]:
val_df_target = val_df[target]
val_df = val_df.drop(target, axis=1)
val_df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
102316,237fc38cd7a5dcfb,"A tag has been placed on Hillsboro Aviation, Inc., requesting that it be speedily deleted from Wikipedia. This has been done under the criteria for speedy deletion, because the article seems to be blatant advertising which only promotes a company, product, group or service and which would need to be fundamentally rewritten in order to become an encyclopedia article. Please read the general criteria for speedy deletion, particularly item 11, as well as the guidelines on spam. \n\nIf you can indicate why the subject of this article is not blatant advertising, you may contest the tagging. To do this, please add on the top of the page and leave a note on the article's talk page explaining your position. Please do not remove the speedy deletion tag yourself, but don't hesitate to add information to the article that would help make it encyclopedic, as well as adding any citations from reliable sources to ensure that the article will be verifiable. Feel free to leave a note on my talk pa...",0.00,0.0,0.00,0.0,0.00,0.0
31494,53a352b1bb6f4ff9,"I'm willing to take criticism for my edits, and I'm willing to back down if I'm wrong. My statement was in good faith, but it's also true that I made the statement on my own authority without citing a source. However, my statement was really no more weaselly than the statement it was intended to answer. So, I've reverted to my last version (because I have criticisms of ImpuMozhi's edits that I think are valid) but removed both of the weasel statements (because I concede that your criticism of some of my wording is valid). If you believe that I was impolite or uncivil to ImpuMozhi or anyone else, then please let me know what I did wrong so that I may tender my apology.",0.00,0.0,0.00,0.0,0.00,0.0
14360,25f225d975b6db01,"""\n\nYou are an open source patent buster. You attacked the Collins patents FIRST, right? I give you facts and you spew politics. I don't have to stand for that. You are being stupid. Stupidity destroys. You are one idiot of many. You are worthless. Apologize for that and I will for mine. But I still hate you. That is a solid fact. There is something wrong with people like you, but that's your problem. \n\nAlso, unlike Hod Lipson's bigoted crazy comments on how """"legal immigrants doing all the work"""" (meaning dirty work as he had referred to) you, Feitas, Merkle Roco (who RUNS the ENTIRE nano department at NSF and he's an X communist scientist!) are taking our GOOD jobs, in fact STEALING THEM as they were stolen from me and as they are stealing the Collins patent. \n\n... Oh, and you said you came here """"15 years ago to do work that no American was able to do."""" What a pompous ass! You think you are BETTER than ANY American living here? Go to hell! """,0.32,0.0,0.00,0.0,0.00,0.0
15076,27db1e4dc8d4f306,"Good. Your encouragement of those who did however was idiotic and self-centered. Too many self-centered arrogant people on this planet. Go away, your big ego is not more important than those people with social anxiety disorder who need help.",0.00,0.0,0.00,0.0,0.64,0.0
76092,cbaed4022a53000e,What is an moitheather no one knows but you a hoe so go eff a duck... ),0.32,0.0,0.16,0.0,0.64,0.0
...,...,...,...,...,...,...,...,...
120607,852589ee2f697076,"COULD YOU JUST LISTEN TO ME FOR 5 SECONDS?!! I know you hate me, and I know you want to leave me alone, but it makes me sad that just for one minor edit, you just block me. I know that's what you have to do. By the way, this is NOT vandalism. Truth. I will leave you alone forever if you give me one more chance. PLEASE!",0.00,0.0,0.00,0.0,0.00,0.0
142797,fbac8e568e73e7e6,"""\nThank you. | Talk """,0.00,0.0,0.00,0.0,0.00,0.0
32417,5640714285ce656c,"""\n\n Why LLD (Mont)? \n\nIn the infobox, Trudeau is titled """"LLD (Mont)"""" which presumably means that he got his LLD from University of Montreal. Is it standard for all Canadian LLDs to 

In [8]:
toxic_ml = regression_blueprints.RegressionBluePrint(datasource=df_train, 
                        target_variable=target,
                        train_split_type='cross',
                        preferred_training_mode='auto',
                        nlp_transformer_columns='comment_text',
                        transformer_chosen='neoyipeng/twitter-roberta-base-sentiment-mlm',
                        transformer_epochs=3,
                        ml_task='regression')
# cardiffnlp/twitter-roberta-base-mar2021
toxic_ml.transformer_settings["train_batch_size"] = 16
toxic_ml.transformer_settings["test_batch_size"] = 16
toxic_ml.transformer_settings["pred_batch_size"] = 8

Ml task is regression
Preferred training mode auto has been chosen. e2eml will automatically detect, if LGBM and Xgboost can use GPU acceleration and optimize the workflow accordingly.


In [9]:
toxic_ml.ml_bp16_regressions_full_processing_bert_transformer()

Started Execute test train split at 18:34:13.
Started Handle rare features at 18:34:13.
Started Sort columns alphabetically at 18:34:13.


Some weights of the model checkpoint at neoyipeng/twitter-roberta-base-sentiment-mlm were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at neoyipeng/twitter-roberta-base-sentiment-mlm and are newly initialized: ['classifi

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32450 entries, 0 to 32449
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   comment_text   32450 non-null  object 
 1   id             32450 non-null  object 
 2   identity_hate  32450 non-null  float64
 3   insult         32450 non-null  float64
 4   obscene        32450 non-null  float64
 5   severe_toxic   32450 non-null  float64
 6   threat         32450 non-null  float64
 7   toxic          32450 non-null  float64
 8   score          32450 non-null  float64
 9   kfold          32450 non-null  int64  
dtypes: float64(7), int64(1), object(2)
memory usage: 2.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32450 entries, 0 to 32449
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   comment_text   32450 non-null  object 
 1   id             32450 non-null  object 
 2   identi

Some weights of the model checkpoint at neoyipeng/twitter-roberta-base-sentiment-mlm were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at neoyipeng/twitter-roberta-base-sentiment-mlm and are newly initialized: ['classifi

---------------0start-------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32450 entries, 0 to 32449
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   comment_text   32450 non-null  object 
 1   id             32450 non-null  object 
 2   identity_hate  32450 non-null  float64
 3   insult         32450 non-null  float64
 4   obscene        32450 non-null  float64
 5   severe_toxic   32450 non-null  float64
 6   threat         32450 non-null  float64
 7   toxic          32450 non-null  float64
 8   score          32450 non-null  float64
 9   kfold          32450 non-null  int64  
dtypes: float64(7), int64(1), object(2)
memory usage: 2.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32450 entries, 0 to 32449
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   comment_text   32450 non-null  object 
 1   id             

...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32450 entries, 0 to 32449
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   comment_text   32450 non-null  object 
 1   id             32450 non-null  object 
 2   identity_hate  32450 non-null  float64
 3   insult         32450 non-null  float64
 4   obscene        32450 non-null  float64
 5   severe_toxic   32450 non-null  float64
 6   threat         32450 non-null  float64
 7   toxic          32450 non-null  float64
 8   score          32450 non-null  float64
 9   kfold          32450 non-null  int64  
dtypes: float64(7), int64(1), object(2)
memory usage: 2.5+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32450 entries, 0 to 32449
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   comment_text   32450 non-null  object 
 1   id             32450 non-null  object 
 2   identi

Some weights of the model checkpoint at neoyipeng/twitter-roberta-base-sentiment-mlm were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at neoyipeng/twitter-roberta-base-sentiment-mlm and are newly initialized: ['classifi

['/home/thomas/Schreibtisch/Data Science/Preprocessing lib test/model4.pth', '/home/thomas/Schreibtisch/Data Science/Preprocessing lib test/model0.pth', '/home/thomas/Schreibtisch/Data Science/Preprocessing lib test/model3.pth', '/home/thomas/Schreibtisch/Data Science/Preprocessing lib test/model2.pth', '/home/thomas/Schreibtisch/Data Science/Preprocessing lib test/model1.pth']


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warn

The R2 score is 0.7668251735952418
The MAE score is 0.03632107367691607
The Median absolute error score is 0.01262238220622142
The MSE score is 0.06600539522029299
The RMSE score is 0.004356712198187076


In [10]:
from e2eml.full_processing.postprocessing import save_to_production, load_for_production
import gc
# Save pipeline
save_to_production(toxic_ml, file_name='toxic_ml_bert_instance')

# Test on holdout set

In [11]:
toxic_ml.ml_bp16_regressions_full_processing_bert_transformer(val_df)

Started Execute test train split at 20:19:31.
Started Handle rare features at 20:19:31.
Started Sort columns alphabetically at 20:19:31.


Some weights of the model checkpoint at neoyipeng/twitter-roberta-base-sentiment-mlm were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at neoyipeng/twitter-roberta-base-sentiment-mlm and are newly initialized: ['classifi

['/home/thomas/Schreibtisch/Data Science/Preprocessing lib test/model4.pth', '/home/thomas/Schreibtisch/Data Science/Preprocessing lib test/model0.pth', '/home/thomas/Schreibtisch/Data Science/Preprocessing lib test/model3.pth', '/home/thomas/Schreibtisch/Data Science/Preprocessing lib test/model2.pth', '/home/thomas/Schreibtisch/Data Science/Preprocessing lib test/model1.pth']


[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warn

In [12]:
val_y_hat = toxic_ml.predicted_values["nlp_transformer"]
val_y_hat

0      -0.001790
1      -0.002693
2       0.137523
3       0.078491
4       0.184188
          ...   
1995    0.031827
1996   -0.000405
1997    0.000538
1998    0.052142
1999   -0.006482
Name: transformers_mean, Length: 2000, dtype: float32

In [13]:
from sklearn.metrics import mean_absolute_error

# Individual model performance

In [14]:
mae = mean_absolute_error(val_df_target, val_y_hat)
print(mae)

0.03585176950653914
